# Text Generation with RNNs
https://blog.floydhub.com/a-beginners-guide-on-recurrent-neural-networks-with-pytorch/

https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/

http://karpathy.github.io/2015/05/21/rnn-effectiveness/

In [5]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F




### 1 Dataset
Define the path of the file, you want to read and train the model on


In [1]:
'''TODO: set the path of the file'''
path_to_file = 'Shakespeare_karpathy.txt'
text = open(path_to_file, encoding='utf-8').read()


#### Inspect the dataset
Take a look at the first 250 characters in text

In [2]:
print(text[:250])



That, poor contempt, or claim'd thou slept so faithful,
I may contrive our father; and, in their defeated queen,
Her flesh broke me and puttance of expedition house,
And in that same that ever I lament this stomach,
And he, nor Butly and my fury, kno


In [3]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


62 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [6]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [7]:

print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  "'" :   3,
  ',' :   4,
  '-' :   5,
  '.' :   6,
  ':' :   7,
  ';' :   8,
  '?' :   9,
  'A' :  10,
  'B' :  11,
  'C' :  12,
  'D' :  13,
  'E' :  14,
  'F' :  15,
  'G' :  16,
  'H' :  17,
  'I' :  18,
  'J' :  19,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [8]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))



'That, poor co' ---- characters mapped to int ---- > [29 43 36 55  4  1 51 50 50 53  1 38 50]


#### Defining a method to encode one hot labels

In [9]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot


#### Defining a method to make mini-batches for training

In [10]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y




## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [11]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [94]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        self.rnn = nn.RNN(len(self.vocab), self.n_hidden, self.n_layers, batch_first=True, dropout=self.drop_prob)   #batch_first=True -> input and output tensors are provided as (batch, seq, feature)
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))

    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''

        # Passing in the input and hidden state into the model and obtaining outputs
        # x of shape (seq_len, batch, input_size)
        # hidden of shape (num_layers * num_directions, batch, hidden_size)
        out, hidden_t = self.rnn(x, hidden)
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)

        
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden = (torch.zeros(self.n_layers, batch_size, self.n_hidden)

        return hidden

In [235]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        self.lstm = nn.LSTM(len(self.vocab), self.n_hidden, self.n_layers, dropout=self.drop_prob, batch_first=True)
        # add Dropout layer?
        self.fc = nn.Linear(self.n_hidden, len(self.vocab))
        # add sigmoid layer?


    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''
        #print('lstm in', x.shape, hidden.shape)
        cell = self.init_hidden(hidden.shape[1])
        out, (hidden_t, cell_t)= self.lstm(x, (hidden, cell))
        #print('lstm out', out.shape, hidden_t.shape)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        #print('linear out', out.shape)
       
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        hidden = torch.zeros(self.n_layers, batch_size, self.n_hidden).cuda()
        # cell state should be included as for now the model only backpropagates the loss over the hidden states and not the cell states
        return hidden



#### Declaring the train method


train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [229]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            # zero accumulated gradients
            model.zero_grad()
            #print('inputs', inputs.shape, torch.cat(h).reshape([model.n_layers,batch_size,-1]).cuda().shape)
            '''TODO: feed the current input into the model and generate output'''
            output, h = model(inputs, torch.cat(h).reshape([model.n_layers,batch_size,-1]).cuda()) 
            '''TODO: compute the loss!'''
            loss = criterion(output, targets.view(-1))
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()
                    #print('Feed2')
                    '''TODO: feed the current input into the model and generate output'''
                    output, val_h = model(inputs, torch.cat(val_h).reshape([model.n_layers,batch_size,-1]).cuda())

                    '''TODO: compute the validation loss!'''
                    val_loss = criterion(output, targets.view(-1))

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                '''TODO: sample from the model to generate texts'''
                input_eval = 'Dear'
                print(sample(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [182]:
def predict(model, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''

    # tensor inputs
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    #print('predict')
    #print('h',len(h))
    #print('h[0]', h[0].shape)
    #print('Feed3')
    '''TODO: feed the current input into the model and generate output'''
    #print(torch.cat(h).reshape([model.n_layers,1,-1]).cuda().shape)
    output, h = model(inputs, torch.cat(h).reshape([model.n_layers,1,-1]).cuda())

    # get the character probabilities
    p = F.softmax(output, dim=1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [176]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    #print('sample')
    #print('h',len(h))
    #print('h[0]', h[0].shape)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
      '''TODO: pass in the previous character and get a new one'''
      char, h = predict(model, char, h, top_k=top_k)
      chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [236]:
''''TODO: Try changing the number of units in the network to see how it affects performance'''
n_hidden = 256
n_layers = 2

vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)
lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

VanillaCharRNN(
  (rnn): RNN(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)
LSTMCharRNN(
  (lstm): LSTM(62, 256, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=256, out_features=62, bias=True)
)



###### Declaring the hyperparameters

In [216]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 10
seq_length = 50
n_epochs = 2  # start smaller if you are just testing initial behavior


##### Train the model and have fun with the generated texts

In [217]:

''''TODO: compare the results from the vanilla and LSTM model'''


"'TODO: compare the results from the vanilla and LSTM model"

In [218]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [237]:
train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/2... Step: 50... Loss: 3.3567... Val Loss: 3.2897
Dearreai  
t ersr
a roreieai noeiresnnaeairraiisooetonsn
rassei
ro sa
os  
tt oeeires
tn ste i aree sneiann srs
n srr sr
 

s
se
snaen
tionooee   r   e  ts aee
ri or aa r n n r n r

iinoot
tnost
nar
arn
rs r
rerss it etreeaerat oasrro an
n
e tninno
nsnnrtsa tsttt
a t 
 
ta  itooa 
 stntn
 ssossorro i inrit rsio  nr
 r eraioa ta
 oeeor  e
sssar e
sasae arit
rotrnaen srsatrn
 osrn sna  
rrie ne

e
t   r
n is o 
e  iensta triae
 roiasenrnsit

esote n 
neet totniir etr eir  
oooosaareoo   reiennorn  nn osoro
etnnto   s ni
sotoseii
rtea te
  aoa aataraie
sera  r 
in  a 
anoonr e tiostei
ot
 oas
osnaenteiesieotra iees rssere
ae
erait 

rt se
eeiorrins
eteio
t r
taar aieoion
ora
ear ot ono 
oattria
easa  ss at  aasr
ite
renooa aoe
 as to
oteitiaeaer rtnreataase atiir aatiaiai
e ste atsrot
aaisi
n ei et 
s o atos  aet oeean isn issaoin a
eitre
n  sas  
orroa aeant ii
eanrs inanstinstntrnr s
ea tisi
oee
oai
ersitsn e  ones naar enr i aa

##### Generate text

In [213]:
print(sample(vanilla_model, 100, prime='The', top_k=10))

Then have by my liget
To stop the plory:
But see, the what you, thou loyous my with our should so lord; 


In [212]:
print(sample(lstm_model, 100, prime='The', top_k=10))

Thessaf sur.
Alane braid bringram.
Time, thotels of yourdlace potim the condew a kind was
A bran sore si
